In [41]:
import numpy as np
import pandas as pd 
import keras
import re
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping,ModelCheckpoint
# fix random seed for reproducibility
np.random.seed(26)

In [20]:
work_train= {} 
work_test = {}
pre_process=["tfidf_tsvd_100.csv","w2v_100.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"nw_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"nw_working_test_"+f)

In [8]:
y=pd.read_csv("../bases/new_training_variants.csv").Class.values - 1

In [9]:
magic_feat=pd.read_csv("../l2_meta_features/svd25_molecular_functions.csv")
magic_feat=magic_feat.drop(["Class","Gene","ID","Variation"],axis=1)
magic_train=magic_feat[:len(y)]
magic_test=magic_feat[len(y):]
magic_train=magic_train.reset_index(drop=True)
magic_test=magic_test.reset_index(drop=True)

In [22]:
ID_test=work_test["w2v_100"]["ID"]
ID_train=work_train["w2v_100"]["ID"]

In [28]:
deep_train=work_train["tfidf_tsvd_100"]#.merge(work_train["w2v_100"])
deep_train=deep_train.drop("ID",axis=1)
deep_test=work_test["tfidf_tsvd_100"]#.merge(work_test["w2v_100"])
deep_test=deep_test.drop("ID",axis=1)

In [29]:
deep_train_1=pd.concat((deep_train,magic_train),axis=1)
deep_test_1=pd.concat((deep_test,magic_test),axis=1)

In [ ]:
deep_train_1[]

In [48]:
deep_train_1.columns

Index(['Substitutions_var', 'Stop_codon_var', 'Fusion_var', 'gene_fusion_var',
       'Deletion_var', 'del_or_ins_var', 'Amplification_var', 'Truncation_var',
       'exon_var', 'frameshift_var',
       ...
       'molecular_SVD_16', 'molecular_SVD_17', 'molecular_SVD_18',
       'molecular_SVD_19', 'molecular_SVD_20', 'molecular_SVD_21',
       'molecular_SVD_22', 'molecular_SVD_23', 'molecular_SVD_24',
       'molecular_SVD_25'],
      dtype='object', length=162)

In [50]:
deep_test_1.columns

Index(['Substitutions_var', 'Stop_codon_var', 'Fusion_var', 'gene_fusion_var',
       'Deletion_var', 'del_or_ins_var', 'Amplification_var', 'Truncation_var',
       'exon_var', 'frameshift_var',
       ...
       'molecular_SVD_16', 'molecular_SVD_17', 'molecular_SVD_18',
       'molecular_SVD_19', 'molecular_SVD_20', 'molecular_SVD_21',
       'molecular_SVD_22', 'molecular_SVD_23', 'molecular_SVD_24',
       'molecular_SVD_25'],
      dtype='object', length=162)

In [56]:
deep_train_1.shape

(3689, 162)

In [64]:
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=26)
cvscores = []
for train, test in kfold.split(np.array(deep_train_1), y):
  # create model
    model = Sequential()
    model.add(Dense(30, input_dim=162, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30,activation="relu"))
    model.add(Dense(9, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    es=EarlyStopping(monitor="loss",mode="min",patience=100)
    checkpoint=ModelCheckpoint(filepath="checknn",save_best_only=True,monitor="val_loss",mode="min")
    model.fit(np.array(deep_train_1)[train], y_cat[train],epochs=100,batch_size=50,validation_split=0.2,callbacks=[es,checkpoint],verbose=True)
    model.load_weights("checknn")
    # evaluate the model
    scores = model.evaluate(np.array(deep_train_1)[test], y_cat[test])
    print("%s: %.2f" % (model.metrics_names[0], scores[0]))
    cvscores.append(scores[0])
print("%.2f (+/- %.2f)" % (np.mean(cvscores), np.std(cvscores)))

Train on 2356 samples, validate on 590 samples
Epoch 1/100
2356/2356 [==============================] - 2s - loss: 12.8008 - acc: 0.2003 - val_loss: 10.1937 - val_acc: 0.3661
Epoch 2/100
2356/2356 [==============================] - 0s - loss: 11.9090 - acc: 0.2585 - val_loss: 10.1936 - val_acc: 0.3661
Epoch 3/100
2356/2356 [==============================] - 0s - loss: 11.9060 - acc: 0.2602 - val_loss: 10.1935 - val_acc: 0.3661
Epoch 4/100
2356/2356 [==============================] - 0s - loss: 11.9353 - acc: 0.2568 - val_loss: 10.1935 - val_acc: 0.3661
Epoch 5/100
2356/2356 [==============================] - 0s - loss: 12.7862 - acc: 0.2029 - val_loss: 13.1713 - val_acc: 0.1814
Epoch 6/100
2356/2356 [==============================] - 0s - loss: 13.1121 - acc: 0.1842 - val_loss: 13.1713 - val_acc: 0.1814
Epoch 7/100
2356/2356 [==============================] - 0s - loss: 12.7104 - acc: 0.2076 - val_loss: 13.1712 - val_acc: 0.1814
Epoch 8/100
2356/2356 [==============================] - 

2356/2356 [==============================] - 0s - loss: 11.8191 - acc: 0.2666 - val_loss: 10.1988 - val_acc: 0.3661
Epoch 65/100
2356/2356 [==============================] - 0s - loss: 11.8186 - acc: 0.2666 - val_loss: 10.1989 - val_acc: 0.3661
Epoch 66/100
2356/2356 [==============================] - 0s - loss: 11.8101 - acc: 0.2670 - val_loss: 10.1992 - val_acc: 0.3661
Epoch 67/100
2356/2356 [==============================] - 0s - loss: 11.8236 - acc: 0.2661 - val_loss: 10.1994 - val_acc: 0.3661
Epoch 68/100
2356/2356 [==============================] - 0s - loss: 11.8045 - acc: 0.2666 - val_loss: 10.1996 - val_acc: 0.3661
Epoch 69/100
2356/2356 [==============================] - 0s - loss: 11.8217 - acc: 0.2661 - val_loss: 10.2000 - val_acc: 0.3661
Epoch 70/100
2356/2356 [==============================] - 0s - loss: 11.7968 - acc: 0.2678 - val_loss: 10.2003 - val_acc: 0.3661
Epoch 71/100
2356/2356 [==============================] - 0s - loss: 11.8307 - acc: 0.2653 - val_loss: 10.1947

2359/2359 [==============================] - 0s - loss: 12.3054 - acc: 0.2170 - val_loss: 14.1272 - val_acc: 0.1169
Epoch 27/100
2359/2359 [==============================] - 0s - loss: 12.5257 - acc: 0.1908 - val_loss: 13.1476 - val_acc: 0.1780
Epoch 28/100
2359/2359 [==============================] - 0s - loss: 12.6810 - acc: 0.1874 - val_loss: 14.1750 - val_acc: 0.1169
Epoch 29/100
2359/2359 [==============================] - 0s - loss: 12.3333 - acc: 0.2166 - val_loss: 14.1713 - val_acc: 0.1169
Epoch 30/100
2359/2359 [==============================] - 0s - loss: 12.2878 - acc: 0.2217 - val_loss: 14.1755 - val_acc: 0.1169
Epoch 31/100
2359/2359 [==============================] - 0s - loss: 12.2292 - acc: 0.2200 - val_loss: 13.9495 - val_acc: 0.1169
Epoch 32/100
2359/2359 [==============================] - 0s - loss: 12.0650 - acc: 0.2018 - val_loss: 14.1194 - val_acc: 0.1169
Epoch 33/100
2359/2359 [==============================] - 0s - loss: 11.6804 - acc: 0.2120 - val_loss: 11.9194

2359/2359 [==============================] - 0s - loss: 1.8339 - acc: 0.2654 - val_loss: 1.7604 - val_acc: 0.3746
Epoch 91/100
2359/2359 [==============================] - 0s - loss: 1.8299 - acc: 0.2654 - val_loss: 1.7430 - val_acc: 0.3746
Epoch 92/100
2359/2359 [==============================] - 0s - loss: 1.8360 - acc: 0.2645 - val_loss: 1.7698 - val_acc: 0.3746
Epoch 93/100
2359/2359 [==============================] - 0s - loss: 1.8336 - acc: 0.2649 - val_loss: 1.7635 - val_acc: 0.3746
Epoch 94/100
2359/2359 [==============================] - 0s - loss: 1.8354 - acc: 0.2654 - val_loss: 1.7723 - val_acc: 0.3746
Epoch 95/100
2359/2359 [==============================] - 0s - loss: 1.8371 - acc: 0.2649 - val_loss: 1.7644 - val_acc: 0.3746
Epoch 96/100
2359/2359 [==============================] - 0s - loss: 1.8332 - acc: 0.2654 - val_loss: 1.7732 - val_acc: 0.3746
Epoch 97/100
2359/2359 [==============================] - 0s - loss: 1.8365 - acc: 0.2649 - val_loss: 1.7710 - val_acc: 0.37

2361/2361 [==============================] - 0s - loss: 11.8518 - acc: 0.2609 - val_loss: 9.9949 - val_acc: 0.3773
Epoch 54/100
2361/2361 [==============================] - 0s - loss: 11.7810 - acc: 0.2656 - val_loss: 9.9940 - val_acc: 0.3773
Epoch 55/100
2361/2361 [==============================] - 0s - loss: 11.7963 - acc: 0.2630 - val_loss: 9.9938 - val_acc: 0.3773
Epoch 56/100
2361/2361 [==============================] - 0s - loss: 11.8020 - acc: 0.2639 - val_loss: 9.9890 - val_acc: 0.3773
Epoch 57/100
2361/2361 [==============================] - 0s - loss: 11.8288 - acc: 0.2626 - val_loss: 9.9858 - val_acc: 0.3773
Epoch 58/100
2361/2361 [==============================] - 0s - loss: 11.8575 - acc: 0.2605 - val_loss: 9.9943 - val_acc: 0.3773
Epoch 59/100
2361/2361 [==============================] - 0s - loss: 11.8001 - acc: 0.2639 - val_loss: 9.9876 - val_acc: 0.3773
Epoch 60/100
2361/2361 [==============================] - 0s - loss: 11.8275 - acc: 0.2609 - val_loss: 9.9894 - val_a

2363/2363 [==============================] - 0s - loss: 13.8319 - acc: 0.1384 - val_loss: 12.9675 - val_acc: 0.1912
Epoch 17/100
2363/2363 [==============================] - 0s - loss: 12.6689 - acc: 0.2103 - val_loss: 14.1062 - val_acc: 0.1235
Epoch 18/100
2363/2363 [==============================] - 0s - loss: 12.4920 - acc: 0.2243 - val_loss: 14.1064 - val_acc: 0.1235
Epoch 19/100
2363/2363 [==============================] - 0s - loss: 12.4972 - acc: 0.2243 - val_loss: 14.1066 - val_acc: 0.1235
Epoch 20/100
2363/2363 [==============================] - 0s - loss: 12.4808 - acc: 0.2243 - val_loss: 14.1068 - val_acc: 0.1235
Epoch 21/100
2363/2363 [==============================] - 0s - loss: 12.5057 - acc: 0.2230 - val_loss: 14.1070 - val_acc: 0.1235
Epoch 22/100
2363/2363 [==============================] - 0s - loss: 12.4785 - acc: 0.2239 - val_loss: 14.1069 - val_acc: 0.1235
Epoch 23/100
2363/2363 [==============================] - 0s - loss: 12.4900 - acc: 0.2239 - val_loss: 14.1071

2363/2363 [==============================] - 0s - loss: 12.4827 - acc: 0.2256 - val_loss: 14.1145 - val_acc: 0.1235
Epoch 80/100
2363/2363 [==============================] - 0s - loss: 12.4767 - acc: 0.2256 - val_loss: 14.1138 - val_acc: 0.1235
Epoch 81/100
2363/2363 [==============================] - 0s - loss: 12.4628 - acc: 0.2264 - val_loss: 14.1135 - val_acc: 0.1235
Epoch 82/100
2363/2363 [==============================] - 0s - loss: 12.4829 - acc: 0.2256 - val_loss: 14.1137 - val_acc: 0.1235
Epoch 83/100
2363/2363 [==============================] - 0s - loss: 12.4899 - acc: 0.2247 - val_loss: 14.1131 - val_acc: 0.1235
Epoch 84/100
2363/2363 [==============================] - 0s - loss: 12.4827 - acc: 0.2256 - val_loss: 14.1132 - val_acc: 0.1235
Epoch 85/100
2363/2363 [==============================] - 0s - loss: 12.5168 - acc: 0.2234 - val_loss: 14.1133 - val_acc: 0.1235
Epoch 86/100
2363/2363 [==============================] - 0s - loss: 12.4937 - acc: 0.2247 - val_loss: 14.1137

2364/2364 [==============================] - 0s - loss: 12.4933 - acc: 0.2242 - val_loss: 14.1080 - val_acc: 0.1235
Epoch 42/100
2364/2364 [==============================] - 0s - loss: 12.4969 - acc: 0.2238 - val_loss: 14.1080 - val_acc: 0.1235
Epoch 43/100
2364/2364 [==============================] - 0s - loss: 12.5232 - acc: 0.2217 - val_loss: 14.1080 - val_acc: 0.1235
Epoch 44/100
2364/2364 [==============================] - 0s - loss: 12.5534 - acc: 0.2200 - val_loss: 14.1077 - val_acc: 0.1235
Epoch 45/100
2364/2364 [==============================] - 0s - loss: 12.4906 - acc: 0.2246 - val_loss: 14.1076 - val_acc: 0.1235
Epoch 46/100
2364/2364 [==============================] - 0s - loss: 12.4929 - acc: 0.2242 - val_loss: 14.1077 - val_acc: 0.1235
Epoch 47/100
2364/2364 [==============================] - 0s - loss: 12.4803 - acc: 0.2238 - val_loss: 14.1076 - val_acc: 0.1235
Epoch 48/100
2364/2364 [==============================] - 0s - loss: 12.4716 - acc: 0.2263 - val_loss: 14.1079

In [37]:
y_cat=to_categorical(y,9)